In [3]:
# Seongho Jin
# kaggle quick draw competition

# drawing data를 이미지로 바꿈
# 이미지 전처리
# 좌우대칭, 가우시안 블러 적용 (그림 주변으로 다양한 화소값을 가지게 하기 위해)
# 입력 이미지는 0~1 사이의 값으로 적용

# MobileNetV2 사용하여 학습 (선정이유: weight는 적지만, 높은 성능을 보이는 모델로 캐글 서버에서 학습하기 적합함)



import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))
import glob
from matplotlib import pyplot as plt
import cv2
import keras
import random

['quickdraw-doodle-recognition', 'mydata', 'mydata1']


Using TensorFlow backend.


### DATA arrangement

In [4]:
class_names = sorted([name[:-4] for name in os.listdir('../input/quickdraw-doodle-recognition/train_simplified/')]) # 340 classes
class_dic = {}
for i in range(len(class_names)):
    class_dic[class_names[i]] = i
class_paths = sorted(glob.glob('../input/quickdraw-doodle-recognition/train_simplified/' + "*"))

cols=['drawing', 'key_id', 'recognized', 'word']
total_df = pd.DataFrame(columns=cols)
for c in range(len(class_paths)):
    df = pd.read_csv(class_paths[c], usecols=['drawing', 'key_id', 'recognized', 'word'], nrows=1500)
    df = df[df.recognized == True]
    df = df.head(1000)
    df = df.reset_index(drop=True)
    total_df = total_df.append(df, ignore_index=True)
    print(c)
display(total_df)

# Data divided into train and validation 
# 340 * 1000 = 340,000 data divide into 7:3 ratio

from numpy.random import RandomState
rans = RandomState(seed=0)
train = total_df.sample(frac=0.7, random_state=rans)
val = total_df.loc[~total_df.index.isin(train.index)]
# train.to_csv('train3401000.csv')
# val.to_csv('val3401000.csv')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

,drawing,key_id,recognized,word
0,"[[[0, 22, 37, 64, 255], [218, 220, 227, 228, 2...",5027286841556992,True,The Eiffel Tower
1,"[[[47, 47, 36, 26, 0, 10, 23, 46, 46, 63, 68, ...",5716269791707136,True,The Eiffel Tower
2,"[[[184, 115, 67, 57, 36, 18], [251, 103, 12, 1...",5942899998982144,True,The Eiffel Tower
3,"[[[0, 187, 177, 132, 105, 79, 38, 19, 11], [24...",6226163091374080,True,The Eiffel Tower
4,"[[[0, 21, 43, 83, 97, 158, 169, 172], [162, 16...",4889008825958400,True,The Eiffel Tower
5,"[[[131, 119, 48, 26], [16, 56, 221, 255]], [[1...",5643506171248640,True,The Eiffel Tower
6,"[[[0, 37, 58, 74, 92, 105, 106], [238, 227, 21...",5048121811795968,True,The Eiffel Tower
7,"[[[0, 17, 50, 98, 117, 141, 153, 155], [201, 1...",5715492117413888,True,The Eiffel Tower
8,"[[[0, 20, 28, 48, 57, 57, 66, 57, 55, 65, 85, ...",5672915955613696,True,The Eiffel Tower
9,"[[[0, 12, 31, 70, 107, 118, 127, 137, 150, 161...",6257783634657280,True,The Eiffel Tower


### Data Preprocessing, Augmentation, and Generator

In [5]:
from tensorflow.keras.utils import to_categorical
from imgaug import augmenters as iaa
seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.GaussianBlur(sigma=(0.0, 3.0))
])

BATCH_SIZE = 680
N_CLASS = 340
W, H = 224, 224

LINE_WIDTH = 3
train_path = '../input/mydata1/train3401000.csv'
val_path= '../input/mydata1/val3401000.csv'


def load_random_samples(file, sample_size):
    num_lines = sum(1 for l in open(file))

    skip_idx = random.sample(range(1, num_lines), num_lines - (sample_size + 1))
    samples = pd.read_csv(file, usecols=['drawing',  'word'], 
                       skiprows=skip_idx)
    return samples

def drawing_to_img(drawing,  line_width):
    if type(drawing) == str:
        drawing = eval(drawing)
        
    img = np.zeros((W, H), np.uint8)
    for step in drawing :
        x, y = step
        for i in range(len(x)-1):
            cv2.line(img, (x[i], y[i]), (x[i+1], y[i+1]), 255, line_width) # line으로 연결
    return img


def img_preprocess(img_before):
    img = np.reshape(img_before, (1, W, H))
    img = seq.augment_images(images=img)
    img = np.array(img).astype('float32') / 255
    
    dummy = np.zeros((W, H, 3))
    for i in range(3):
        dummy[:,:,i] = img[0,:,:]
    return dummy

def generator_random(train_path, batch_size=680):
    
    while True:
        df_batch = load_random_samples(train_path, batch_size)

        batch_img = []
        batch_label = []
        for i in range(batch_size):
            img = drawing_to_img(df_batch.drawing[i], LINE_WIDTH)
            img = img_preprocess(img)

            label = class_dic[df_batch.word[i]]
            label = to_categorical(label, N_CLASS)
            
            batch_img.append(img)
            batch_label.append(label)
            
        yield [np.array(batch_img), np.array(batch_label)]

train_gen = generator_random(train_path, BATCH_SIZE)
val_gen = generator_random(val_path, BATCH_SIZE)
x, y = next(train_gen)
print(np.shape(x), np.shape(y))

(680, 224, 224, 3) (680, 340)


### Model

In [6]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Sequential
from keras.layers import GlobalAvgPool2D
from keras.layers import Dense

mobile_model = MobileNetV2(weights='imagenet', input_shape=(224, 224, 3), include_top=False, classes=N_CLASS)
model = Sequential()
model.add(mobile_model)
model.add(GlobalAvgPool2D())
model.add(Dense(N_CLASS, activation='softmax', use_bias=True))
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
9412608/9406464 [==============================] - 0s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 340)               435540    
Total params: 2,693,524
Trainable params: 2,659,412
Non-trainable params: 34,112
_________________________________________________________________


### Training

In [ ]:
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.metrics import categorical_accuracy
from keras.callbacks import ModelCheckpoint
from math import ceil

model.compile(optimizer=Adam(lr=1e-4), loss=categorical_crossentropy, metrics=[categorical_accuracy])

weight_save = 'mobileNet_01.hdf5'
callback = ModelCheckpoint(weight_save,
                           monitor='val_acc',
                           mode='max',
                           save_best_only=True,
                           save_weights_only=True,
                           verbose=1)

n_train = sum(1 for l in open(train_path)) - 1
n_val = sum(1 for l in open(val_path)) - 1
N_EPOCH = 100

history = model.fit_generator(train_gen, 
                                steps_per_epoch=ceil(n_train/BATCH_SIZE),
                                epochs=N_EPOCH,
                                validation_data=val_gen,
                                validation_steps=ceil(n_val/BATCH_SIZE)
                                )


### Test 

In [ ]:
test_path = '../input/quickdraw-doodle-recognition/test_simplified.csv'
testset = pd.read_csv(test_path)

def test_generator(batch_size=680):
    start = 0
    end = 0
    while True:
        end = start + BATCH_SIZE

        batch_img = []
        for i in range(start, end):
            img = drawing_to_img(testset.drawing[i], LINE_WIDTH)
            img = img_preprocess(img)

            batch_img.append(img)
        
        start += end
        yield np.array(batch_img)


n_test = sum(1 for l in open(val_test)) - 1
test_gen = test_generator(test_path, BATCH_SIZE)

output = model.predict_generator(test_gen, steps=ceil(n_test/BATCH_SIZE), verbose=1)

result = pd.DataFrame()
for i in range(n_test):

    output_dic = {}
    for i, o in enumerate(output[i]):
        output_dic[o] = i

    pred = []
    for j, o in enumerate(sorted(output[i], reverse=True)):
        pred.append(class_names[output_dic[o]])
        if j == 2:
            break
    
#     print(testset[i], end='')
#     for j in range(3):
#         print(pred[j], end='')
#     print()
#     break
    
    result.append(testset[i], pred)

result.to_csv('submission.csv')